In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, FunctionMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here


In [2]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

#data_raw = pd.read_csv('/home/sftpuser1/uploads/data.csv')
#data_raw

no docx
no docx


In [3]:
data_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=2038294867', 
    4)
data_raw

,link,news_date,news_text,news_topic
0,https://social.evraz.com/news/news/itogi-grant...,2024-06-14 00:00:00,,Итоги грантового конкурса социальных проектов ...
1,https://www.forbes.com/sites/reneehsia/2024/06...,2024-06-14 00:00:00,Health insurance doesn't protect you from rece...,Have Good Health Insurance? Too Bad. You Could...
2,https://www.forbes.com/sites/brianmazique/2024...,2024-06-14 00:00:00,Conor McGregor is officially off the UFC 303 c...,Conor McGregor Is Out And UFC 303 Main Card Ge...
3,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14 00:00:00,"<p>In its first test in Malawi, an AI app is h...",The AI Revolution Comes for Farmers Growing a ...
4,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14 00:00:00,The price of South Korea’s semiconductor expor...,AI Demand Helps Korean Chip Export Prices to R...
...,...,...,...,...
69,https://www.interfax.ru/russia/966536,2024-06-14 00:00:00,Массовая атака дронов отражена в Ростовской об...,Силы ПВО отразили массовую атаку беспилотников...
70,https://www.interfax.ru/moscow/966535,2024-06-14 00:00:00,Площадь пожара составила 800 кв. м источник: И...,Кровля горящего здания обрушилась на севере Мо...
71,https://www.interfax.ru/world/966533,2024-06-14 00:00:00,Постоянный комитет антинеоколониального Движен...,Медведев анонсирует учреждение комитета антине...
72,https://www.interfax.ru/world/966532,2024-06-14 00:00:00,На борту начался пожар источник: Интерфакс,Запущенные хуситами ракеты попали по судну в А...


In [4]:
#df = data_raw.copy()
#df['article_time'] = pd.to_datetime(df['article_time'])
#filtered_df = df[df['article_time'] == '2024-05-29']
filtered_df = data_raw.copy()

In [5]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

In [6]:
from datetime import datetime
from langchain import hub

def evaluate_news_article(link, news_text, news_date, news_topic):
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
    prompt = hub.pull("mlenparrot/nlmk_grade")
    today = datetime.now().strftime('%Y-%m-%d')
    params = {
        'link': link,
        'news_text': news_text,
        'news_date': news_date,
        'news_topic': news_topic,
        'today': today
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

result = evaluate_news_article(
    link='https://example.com/news-article',
    news_text='This is a sample news text.',
    news_date='2024-06-01',
    news_topic='Sample News Topic'
)

print(result)

{'Source_Authority': 4, 'Relevance_for_NLMK': 3, 'Legislative_Changes': 2, 'Novelty_and_Relevance': 4, 'competitor_extraction': [], 'Events_and_Conferences': 3, 'Competitors_and_Startups': 2, 'Products_and_Technologies': 4, 'Impact_on_the_Industry_and_Market': 3}


In [7]:
def apply_evaluation(row):
    result = evaluate_news_article(
        link=row['link'],
        news_text=row['news_text'],
        news_date=row['news_date'],
        news_topic=row['news_topic']
    )
    return pd.Series(result)

In [8]:
from tqdm import tqdm
tqdm.pandas()
data_news = filtered_df.copy()
evaluation_results = data_news.progress_apply(apply_evaluation, axis=1)

100%|██████████| 74/74 [02:28<00:00,  2.01s/it]


In [9]:
data_news_rated = pd.concat([data_news, evaluation_results], axis=1)#.drop(['reference_time', 'author', 'tags'], axis=1)
data_news_rated

,link,news_date,news_text,news_topic,Source_Authority,Relevance_for_NLMK,Legislative_Changes,Novelty_and_Relevance,competitor_extraction,Events_and_Conferences,Competitors_and_Startups,Products_and_Technologies,Impact_on_the_Industry_and_Market
0,https://social.evraz.com/news/news/itogi-grant...,2024-06-14 00:00:00,,Итоги грантового конкурса социальных проектов ...,3,1,0,4,[],3,2,1,2
1,https://www.forbes.com/sites/reneehsia/2024/06...,2024-06-14 00:00:00,Health insurance doesn't protect you from rece...,Have Good Health Insurance? Too Bad. You Could...,5,0,0,4,[],0,0,0,0
2,https://www.forbes.com/sites/brianmazique/2024...,2024-06-14 00:00:00,Conor McGregor is officially off the UFC 303 c...,Conor McGregor Is Out And UFC 303 Main Card Ge...,5,0,0,4,[],0,0,0,0
3,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14 00:00:00,"<p>In its first test in Malawi, an AI app is h...",The AI Revolution Comes for Farmers Growing a ...,5,2,0,4,[],0,0,4,3
4,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14 00:00:00,The price of South Korea’s semiconductor expor...,AI Demand Helps Korean Chip Export Prices to R...,5,3,0,5,[],0,0,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,https://www.interfax.ru/russia/966536,2024-06-14 00:00:00,Массовая атака дронов отражена в Ростовской об...,Силы ПВО отразили массовую атаку беспилотников...,4,1,0,3,[],0,0,0,2
70,https://www.interfax.ru/moscow/966535,2024-06-14 00:00:00,Площадь пожара составила 800 кв. м источник: И...,Кровля горящего здания обрушилась на севере Мо...,3,0,0,2,[],0,0,0,1
71,https://www.interfax.ru/world/966533,2024-06-14 00:00:00,Постоянный комитет антинеоколониального Движен...,Медведев анонсирует учреждение комитета антине...,4,0,0,3,[],0,0,0,0
72,https://www.interfax.ru/world/966532,2024-06-14 00:00:00,На борту начался пожар источник: Интерфакс,Запущенные хуситами ракеты попали по судну в А...,4,0,0,3,[],0,0,0,2


In [10]:
integer_columns = data_news_rated.select_dtypes(include=['int64']).columns
sum_values = data_news_rated[integer_columns].sum(axis=1)

data_news_rated['rate'] =  sum_values
data_news_rated = data_news_rated.sort_values('rate', ascending=False).reset_index(drop=True) 
data_news_rated

,link,news_date,news_text,news_topic,Source_Authority,Relevance_for_NLMK,Legislative_Changes,Novelty_and_Relevance,competitor_extraction,Events_and_Conferences,Competitors_and_Startups,Products_and_Technologies,Impact_on_the_Industry_and_Market,rate
0,https://rb.ru/news/mistral-ai-invesrments/,2024-06-14 00:00:00,Французский стартап Mistral AI привлек €640 мл...,Главный европейский конкурент OpenAI привлек €...,4,1,0,5,[OpenAI],0,5,5,4,24
1,https://36kr.com/p/2819048418101513?f=rss,2024-06-14 00:00:00,"<p class=""image-wrapper""><img src=""https://img...",10款产品入选，2024夏季「焦点」产品计划重磅发布,3,2,0,4,[],3,2,5,4,23
2,https://36kr.com/newsflashes/2819214534330626?...,2024-06-14 00:00:00,36氪获悉，据商务部官网消息，6月8日，土耳其宣布对原产地为中国的燃油及混合动力乘用车征收4...,商务部新闻发言人就土耳其对我燃油及混合动力乘用车征收额外关税答记者问,3,1,4,5,[],2,0,3,4,22
3,https://36kr.com/p/2817934245677572?f=rss,2024-06-14 00:00:00,<p>36氪获悉，赛桥方舟（深圳）生物科技有限公司（赛桥生物）近期完成近2亿元规模B轮融资，...,专注CGT上游智能装备国产替代，「赛桥生物」获近2亿元B轮融资｜36氪首发,3,1,0,4,[],2,3,5,4,22
4,https://www.bloomberg.com/news/articles/2024-0...,2024-06-14 00:00:00,The price of South Korea’s semiconductor expor...,AI Demand Helps Korean Chip Export Prices to R...,5,3,0,5,[],0,0,5,4,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,https://www.interfax.ru/russia/966544,2024-06-14 00:00:00,6 - над Воронежской и 6 - над Курской источник...,70 дронов уничтожено за ночь над Ростовской об...,3,0,0,4,[],0,0,0,0,7
70,https://www.interfax.ru/russia/966542,2024-06-14 00:00:00,За ночь сбито шесть дронов источник: Интерфакс,Неиспользуемый резервуар нефтебазы в Воронежск...,3,1,0,2,[],0,0,0,1,7
71,https://www.interfax.ru/moscow/966535,2024-06-14 00:00:00,Площадь пожара составила 800 кв. м источник: И...,Кровля горящего здания обрушилась на севере Мо...,3,0,0,2,[],0,0,0,1,6
72,https://www.interfax.ru/world/966543,2024-06-14 00:00:00,"Никто не знает, сколько из захваченных группир...","ХАМАС не знает, сколько израильских заложников...",3,0,0,2,[],0,0,0,0,5


In [11]:
ggl.update_gsheet_with_df(data_news_rated,
                          'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing', 
                          0)

In [12]:
data_news_rated[:3].keys()

Index(['link', 'news_date', 'news_text', 'news_topic', 'Source_Authority',
       'Relevance_for_NLMK', 'Legislative_Changes', 'Novelty_and_Relevance',
       'competitor_extraction', 'Events_and_Conferences',
       'Competitors_and_Startups', 'Products_and_Technologies',
       'Impact_on_the_Industry_and_Market', 'rate'],
      dtype='object')

In [13]:
data_news_summary = data_news_rated[:3][['link', 'news_date', 'news_topic', 'news_text']]
data_news_summary

,link,news_date,news_topic,news_text
0,https://rb.ru/news/mistral-ai-invesrments/,2024-06-14 00:00:00,Главный европейский конкурент OpenAI привлек €...,Французский стартап Mistral AI привлек €640 мл...
1,https://36kr.com/p/2819048418101513?f=rss,2024-06-14 00:00:00,10款产品入选，2024夏季「焦点」产品计划重磅发布,"<p class=""image-wrapper""><img src=""https://img..."
2,https://36kr.com/newsflashes/2819214534330626?...,2024-06-14 00:00:00,商务部新闻发言人就土耳其对我燃油及混合动力乘用车征收额外关税答记者问,36氪获悉，据商务部官网消息，6月8日，土耳其宣布对原产地为中国的燃油及混合动力乘用车征收4...


In [14]:
hub.pull("mlenparrot/nlmk_grade")

StructuredPrompt(input_variables=['link', 'news_date', 'news_text', 'news_topic', 'today'], metadata={'lc_hub_owner': 'mlenparrot', 'lc_hub_repo': 'nlmk_grade', 'lc_hub_commit_hash': '4d478d2ad1871576473da9563261b561da123f5bff1aeabef43a594d60022454'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['today'], template='Какую проблему решаем?\nИТ подразделения НЛМК нуждаются в оперативной информации о новейших продуктах, инициативах конкурентов, инновациях в системах хранения и управления данными, ии, стартапах, законодательных требованиях, конференциях, открытиях и других событиях, которые могут повлиять на стратегию компании.\n\nЧто необходимо сделать?\nВыделение самой важной информации о новых продуктах, инициативах конкурентов, инновациях и других событиях.\n\nЦелевая аудитория\nИТ подразделения НЛМК, руководители и первые лица компаний, заинтересованные в оперативной информации о рынке и конкурентах.\n\nМетрики успеха проекта\nТочность сбора информации.\

In [15]:
def summarize_news_article(counts, news_title, news_text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_summary")
    params = {
        'news_title' : news_title,
        'counts' : counts,
        'news_text' : news_text,
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

In [16]:
pd.set_option('display.max_colwidth', None)
news_text = data_news_summary.loc[0, 'news_text']#.to_string()
news_title = data_news_summary.loc[0, 'news_topic']
pd.reset_option('display.max_colwidth')
news_text

'Французский стартап\xa0Mistral AI привлек €640 млн при оценке\xa0€5,8 млрд. Инвестраунд серии B возглавили General Catalyst и\xa0DST Global. Это следует из данных Crunchbase, на которые обратил внимание RB.RU. источник: RB.RU'

In [17]:
summarize_news_article(500, news_title, news_text)

{'news_topic': 'Главный европейский конкурент OpenAI привлек €640 млн от DST Global, General Catalyst и других',
 'news_summary': 'Французский стартап Mistral AI привлек €640 млн при оценке €5,8 млрд. Инвестраунд серии B возглавили General Catalyst и DST Global. Это следует из данных Crunchbase, на которые обратил внимание RB.RU. источник: RB.RU'}

In [18]:
# Инициализация новых столбцов
data_news_summary['summary'] = ""
data_news_summary['summary_title'] = ""

# Итерация по строкам DataFrame с использованием tqdm
for index, row in tqdm(data_news_summary.iterrows(), total=data_news_summary.shape[0]):
    result = summarize_news_article(500, row['news_topic'], row['news_text'])
    data_news_summary.at[index, 'summary'] = result['news_summary']
    data_news_summary.at[index, 'summary_title'] = result['news_topic']

# Переупорядочивание столбцов для удобства
#data_news_summary = data_news_summary[['link', 'news_date', 'news_topic', 'news_title', 'news_text', 'summary', 'summary_title']]


100%|██████████| 3/3 [00:10<00:00,  3.52s/it]


In [19]:
data_news_summary

,link,news_date,news_topic,news_text,summary,summary_title
0,https://rb.ru/news/mistral-ai-invesrments/,2024-06-14 00:00:00,Главный европейский конкурент OpenAI привлек €...,Французский стартап Mistral AI привлек €640 мл...,Французский стартап Mistral AI привлек €640 мл...,Главный европейский конкурент OpenAI привлек €...
1,https://36kr.com/p/2819048418101513?f=rss,2024-06-14 00:00:00,10款产品入选，2024夏季「焦点」产品计划重磅发布,"<p class=""image-wrapper""><img src=""https://img...","В новость говорится о том, что 10 продуктов бы...",10 продуктов вошли в план «Фокус» на лето 2024...
2,https://36kr.com/newsflashes/2819214534330626?...,2024-06-14 00:00:00,商务部新闻发言人就土耳其对我燃油及混合动力乘用车征收额外关税答记者问,36氪获悉，据商务部官网消息，6月8日，土耳其宣布对原产地为中国的燃油及混合动力乘用车征收4...,Согласно сообщению на сайте Министерства комме...,Пресс-секретарь Министерства коммерции ответил...


In [20]:
ggl.update_gsheet_with_df(data_news_summary,
                          'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing', 
                        1)

In [21]:
data_news_summary.loc[0,'summary']

'Французский стартап Mistral AI привлек €640 млн при оценке €5,8 млрд. Инвестраунд серии B возглавили General Catalyst и DST Global. Это следует из данных Crunchbase, на которые обратил внимание RB.RU. источник: RB.RU'